In [1]:
import lightgbm as lgb

In [2]:
#导入需要的包
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from operator import itemgetter
import datetime
%matplotlib inline
import lightgbm as lgb

#导入数据
df_train_data=pd.read_csv('used_car_train_20200313.csv',sep=' ')
# df_test_data=pd.read_csv('used_car_testB_20200421.csv',sep=' ')
print(df_train_data.shape)
# print(df_test_data.shape)

(150000, 31)


In [3]:
#数据备份
df_train=df_train_data.copy()
#查看总体数据
df_train.head(5)

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


In [4]:
#查看训练集的变量数据类型
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             150000 non-null  int64  
 1   name               150000 non-null  int64  
 2   regDate            150000 non-null  int64  
 3   model              149999 non-null  float64
 4   brand              150000 non-null  int64  
 5   bodyType           145494 non-null  float64
 6   fuelType           141320 non-null  float64
 7   gearbox            144019 non-null  float64
 8   power              150000 non-null  int64  
 9   kilometer          150000 non-null  float64
 10  notRepairedDamage  150000 non-null  object 
 11  regionCode         150000 non-null  int64  
 12  seller             150000 non-null  int64  
 13  offerType          150000 non-null  int64  
 14  creatDate          150000 non-null  int64  
 15  price              150000 non-null  int64  
 16  v_

In [5]:
#查看唯一的字符串类型变量的数值分布
df_train['notRepairedDamage'].value_counts()

notRepairedDamage
0.0    111361
-       24324
1.0     14315
Name: count, dtype: int64

In [6]:
#查看训练集中offerType变量
df_train['offerType'].value_counts()

offerType
0    150000
Name: count, dtype: int64

In [7]:
#查看训练集seller变量
df_train['seller'].value_counts()

seller
0    149999
1         1
Name: count, dtype: int64

In [8]:
#获得需要的列名
numeric_columns=df_train.select_dtypes(exclude='object').columns
columns=[col for col in numeric_columns if col not in ['SaleID', 'name', 'regDate',
                                         'regionCode','seller', 'offerType', 'creatDate']]
#根据列名提取数据
train_set=df_train[columns]
#计算各列于交易价格的相关性
corr_matrix=train_set.corr()
corr_matrix['price'].sort_values()

v_3         -0.730946
kilometer   -0.440519
v_11        -0.275320
v_10        -0.246175
v_9         -0.206205
v_4         -0.147085
v_7         -0.053024
brand       -0.043799
v_13        -0.013993
v_14         0.035911
v_1          0.060914
v_6          0.068970
v_2          0.085322
model        0.136983
v_5          0.164317
fuelType     0.200536
power        0.219834
bodyType     0.241303
gearbox      0.329075
v_0          0.628397
v_8          0.685798
v_12         0.692823
price        1.000000
Name: price, dtype: float64

In [9]:
len(df_train['model'].unique())   #结果为249
len(df_train['brand'].unique())    #结果为40
len(df_train['bodyType'].unique()) #结果为9
len(df_train['fuelType'].unique()) #结果为8

8

In [10]:
#将nan填充为-1
df_train=df_train.fillna(-1)

In [11]:
#获得model所有属性值，除nan
model_list=[model for model in list(df_train['model'].unique()) if model != -1]

In [12]:
#将所有的属性值切分出来，用1和0表示是否属于该属性，并保留nan值
all_info={}
for n in range(len(model_list)):
    # print(n)
    info=[]
    for i in range(len(df_train['model'])):
        if df_train['model'][i]==model_list[n]:
            info.append(1)
        elif df_train['model'][i]==-1:
            info.append(np.nan)
        else:
            info.append(0)
    all_info['model'+str(model_list[n])]=info

In [13]:
#将切分出来的属性保存为csv文件
df_info=pd.DataFrame(all_info)
df_info.to_csv('model.csv',encoding='utf_8_sig')

In [14]:
#获得除nan外的所有属性值列表
brand_list=[brand for brand in list(df_train['brand'].unique()) if brand != -1]
body_list=[bodyType for bodyType in list(df_train['bodyType'].unique()) if bodyType != -1]
fuel_list=[fuelType for fuelType in list(df_train['fuelType'].unique()) if fuelType != -1]

#将brand所有的属性值切分出来，用1和0表示是否属于该属性，并保留nan值
all_info={}
for n in range(len(brand_list)):
    #print(n)
    info=[]
    for i in range(len(df_train['brand'])):
        if df_train['brand'][i]==brand_list[n]:
            info.append(1)
        elif df_train['brand'][i]==-1:
            info.append(np.nan)
        else:
            info.append(0)
    all_info['brand'+str(brand_list[n])]=info

#将bodyType所有的属性值切分出来，用1和0表示是否属于该属性，并保留nan值
for n in range(len(body_list)):
    #print(n)
    info=[]
    for i in range(len(df_train['bodyType'])):
        if df_train['bodyType'][i]==body_list[n]:
            info.append(1)
        elif df_train['bodyType'][i]==-1:
            info.append(np.nan)
        else:
            info.append(0)
    all_info['bodyType'+str(body_list[n])]=info

#将fuelType所有的属性值切分出来，用1和0表示是否属于该属性，并保留nan值
for n in range(len(fuel_list)):
    # print(n)
    info=[]
    for i in range(len(df_train['fuelType'])):
        if df_train['fuelType'][i]==fuel_list[n]:
            info.append(1)
        elif df_train['fuelType'][i]==-1:
            info.append(np.nan)
        else:
            info.append(0)
    all_info['fuelType'+str(fuel_list[n])]=info

#将brand、bodyType和fuelType的结果输出为csv文件
df_info=pd.DataFrame(all_info)
df_info.to_csv('brand_body_fuel.csv',encoding='utf_8_sig')

In [15]:
#处理字符串属性的列
new_col=[]
for n in range(len(df_train['notRepairedDamage'])):
    try:
        new_col.append(int(float(df_train['notRepairedDamage'][n])))
    except:
        new_col.append(np.nan)
df_train['notRepairedDamage_new']=new_col

In [16]:
#构建使用时间
df_train['used_time'] = (pd.to_datetime(df_train['creatDate'], format='%Y%m%d', errors='coerce') - 
                            pd.to_datetime(df_train['regDate'], format='%Y%m%d', errors='coerce')).dt.days

In [17]:
# 特征筛选

In [18]:
#导入构建的变量数据
df_model_data=pd.read_csv('model.csv')
df_other_data=pd.read_csv('brand_body_fuel.csv')

#加入交易价格一列
df_model=df_model_data.copy()
df_other=df_other_data.copy()
df_model['price']=df_train['price']
df_other['price']=df_train['price']
#去掉空值行，以计算相关系数
df_model.dropna(inplace=True)
df_other.dropna(inplace=True)

#计算相关系数
cor=[]
variable=[]
for column in df_model.columns:
    cor.append(np.corrcoef(df_model[column],df_model['price'])[0,1])
    variable.append(column)
for column in df_other.columns:
    cor.append(np.corrcoef(df_other[column],df_other['price'])[0,1])
    variable.append(column)

#导出文件
info={}
info['特征']=variable
info['相关性']=cor
info_df=pd.DataFrame(info)
info_df.to_csv('corcoef.csv',encoding='utf_8_sig')

#根据相关系数帅选变量，阈值0.1
info_df=pd.read_csv('corcoef.csv')
df_new_var=pd.DataFrame(df_train['SaleID'])
for n in range(len(info_df['相关性'])):
    corcoef=info_df['相关性'][n]
    if np.abs(corcoef)>=0.1:
        new_variable=info_df['特征'][n]
        if new_variable in df_model_data.columns:
            df_new_var[new_variable]=df_model_data[new_variable]
        elif new_variable in df_other_data.columns:
            df_new_var[new_variable]=df_other_data[new_variable]

#将未被删除的新构造的变量加入到数据中
df_train=df_train.merge(df_new_var, how='left',on='SaleID')

In [19]:
# 根据lightGBM模型进行帅选

In [21]:
#切分特征和标签

train_set=df_train.copy()
x_train=train_set[[col for col in train_set.columns if col not in ['SaleID','name','regDate','notRepairedDamage','regionCode','creatDate',
                                                                 'price','model','bodyType','fuelType','gearbox','brand']]]
y_train=train_set['price']

features = pd.get_dummies(x_train)
feature_names = list(features.columns)
features = np.array(features)
labels = np.array(y_train).reshape((-1, ))
feature_importance_values = np.zeros(len(feature_names))
task='regression'
early_stopping=False
eval_metric= 'l2'
n_iterations=10
for _ in range(n_iterations):
    if task == 'classification':
        model = lgb.LGBMClassifier(n_estimators=1000, learning_rate = 0.05, verbose = -1)
    elif task == 'regression':
        model = lgb.LGBMRegressor(n_estimators=1000, learning_rate = 0.05, verbose = -1)
    else:
        raise ValueError('Task must be either "classification" or "regression"')
 # If training using early stopping need a validation set
    if early_stopping:
        train_features, valid_features, train_labels, valid_labels = train_test_split(features, labels, test_size = 0.15)
  # Train the model with early stopping
        model.fit(train_features, train_labels, eval_metric = eval_metric,eval_set = [(valid_features, valid_labels)],early_stopping_rounds = 100, verbose = -1)
        gc.enable()
        del train_features, train_labels, valid_features, valid_labels
        gc.collect()
  
    else:
        model.fit(features, labels)
  # Record the feature importances
    feature_importance_values += model.feature_importances_ / n_iterations
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})

        

In [22]:
#按照重要性大小对特征进行排序
feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index(drop = True)
#计算特征的相对重要性，全部特征的相对重要性之和为1
feature_importances['normalized_importance'] = feature_importances['importance'] / feature_importances['importance'].sum()
#计算特征的累计重要性
feature_importances['cumulative_importance'] = np.cumsum(feature_importances['normalized_importance'])
#选取累计重要性大于0.99的特征，这些特征将会被删除掉。
drop_columns=list(feature_importances.query('cumulative_importance>0.99')['feature'])


In [23]:
#去掉重要度低的列
x_set=x_train.copy()
x_set.drop(drop_columns,axis=1,inplace=True)
#对数据集总体概览
x_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   power                  150000 non-null  int64  
 1   kilometer              150000 non-null  float64
 2   v_0                    150000 non-null  float64
 3   v_1                    150000 non-null  float64
 4   v_2                    150000 non-null  float64
 5   v_3                    150000 non-null  float64
 6   v_4                    150000 non-null  float64
 7   v_5                    150000 non-null  float64
 8   v_6                    150000 non-null  float64
 9   v_7                    150000 non-null  float64
 10  v_8                    150000 non-null  float64
 11  v_9                    150000 non-null  float64
 12  v_10                   150000 non-null  float64
 13  v_11                   150000 non-null  float64
 14  v_12                   150000 non-nu

In [24]:
x_set.fillna(1,inplace=True)

In [25]:
# 模型预测

In [26]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(x_set,y_train)


LinearRegression()

In [27]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
def model_goodness(model,x,y):
    prediction=model.predict(x)
    mae=mean_absolute_error(y,prediction)
    mse=mean_squared_error(y,prediction)
    rmse=np.sqrt(mse)
    print('MAE:',mae)
    print('MSE:',mse)
    print('RMSE:',rmse)

In [28]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
        print("Scores:", scores)    
        print("Mean:", scores.mean())
        print("Standard deviation:", scores.std())

In [30]:
model_goodness(lin_reg,x_set,y_train)

MAE: 2630.0521705073293
MSE: 17881063.298454825
RMSE: 4228.6006312319005


In [31]:
from sklearn.ensemble import RandomForestRegressor
forest_reg=RandomForestRegressor()
forest_reg.fit(x_set,y_train)

RandomForestRegressor()

In [32]:
model_goodness(forest_reg,x_set,y_train)

MAE: 226.34605604461765
MSE: 277970.42329903576
RMSE: 527.2290046071402


In [33]:
scores=cross_val_score(forest_reg,x_set,y_train,scoring='neg_mean_absolute_error',cv=10)
mae_scores=np.abs(-scores)
display_scores(mae_scores)

Scores: [612.99595626 613.48300304 609.24658375 613.88981985 624.64238799
 611.72145174 607.64801113 607.97352595 613.34913604 621.84416341]
Mean: 613.6794039160352
Standard deviation: 5.290818053733389


In [40]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt=GradientBoostingRegressor()
gbrt.fit(x_set,y_train)

GradientBoostingRegressor()

In [41]:
scores=cross_val_score(gbrt,x_set,y_train,scoring='neg_mean_absolute_error',cv=10)
mae_scores=np.abs(scores)
display_scores(mae_scores)

Scores: [869.19951724 876.63340014 869.05393992 872.21075463 896.17127288
 871.47156496 876.88429914 866.85936771 885.38244894 894.50093935]
Mean: 877.8367504916703
Standard deviation: 10.087381181927329


In [38]:
import lightgbm as lgb
import xgboost as xgb
xgb_reg= xgb.XGBRegressor()
xgb_reg.fit(x_set,y_train)

In [39]:
scores=cross_val_score(xgb_reg,x_set,y_train,scoring='neg_mean_absolute_error',cv=10)
mae_scores=np.abs(scores)
display_scores(mae_scores)

NameError: name 'X_train_s' is not defined

In [ ]:
lgb_reg=lgb.LGBMRegressor()
lgb_reg.fit(x_train,y_train)

In [ ]:
scores=cross_val_score(lgb_reg,x_set,y_train,scoring='neg_mean_absolute_error',cv=10)
mae_scores=np.abs(scores)
display_scores(mae_scores)